In [1]:
#@ RETORNA UM XLSX COM AS VIDAS ATIVAS DE UM CONJUNTO DE EMPRESAS (RAIA DROGASIL)
import requests
import pandas as pd
import io

url = 'https://ws1.soc.com.br/WebSoc/exportadados?parametro='

empresas = ['1379598'] #RAIA DROGASIL
tipos = ['2','3']
empresa = 'RAIA_DROGASIL'
dataframes=[]

for tipo in tipos:
    for item in empresas:
        body = f'''{{'empresa':'{item}',
                    'codigo':'11672',
                    'chave':'8abb9e50b4201b1cbfd5',
                    'tipoSaida':'json',
                    'tipoBusca':'0',
                    'sequencialFicha':'',
                    'cpfFuncionario':'',
                    'filtraPorTipoSocged':'1',
                    'codigoTipoSocged':'{tipo}',
                    'dataInicio':'',
                    'dataFim':'',
                    'dataEmissaoInicio':'11/03/2024',
                    'dataEmissaoFim':'13/03/2024'}}'''
                    
    response = requests.post(f'{url}{body}')
    if response.status_code == 200:
        try:
            data = io.StringIO(response.text)
            df = pd.read_json(data)
            dataframes.append(df)
        except ValueError as e:
            print(f'erro {e}')
    else:
        print('Erro Chamada')

final_df = pd.concat(dataframes, ignore_index=True)
final_df.to_excel(f'vidas_{empresa}.xlsx', index=False)

In [2]:
filtered_df = final_df[final_df['OBSERVACAO'].str.contains('MIGRACAO PORTO SEGURO>BENCORP: 11/03/2024', na=False)]
filtered_df['INDEX_CREATED'] = filtered_df['NM_GED'].str.extract(r'(\d+)_')
df2 = pd.read_json('D:\\Bencorp Projex\\MIGRACAO_RAIA_DROGASIL\\py_upload_ged_scripts\\upload_ae.json',orient='records')
filtered_df['INDEX_CREATED'] = filtered_df['INDEX_CREATED'].astype(str)
df2['INDEX_IMAGEM'] = df2['INDEX_IMAGEM'].astype(str)
final_df = df2[~df2['INDEX_IMAGEM'].isin(filtered_df['INDEX_CREATED'])]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13528\2905162572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['INDEX_CREATED'] = filtered_df['NM_GED'].str.extract(r'(\d+)_')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13528\2905162572.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['INDEX_CREATED'] = filtered_df['INDEX_CREATED'].astype(str)


In [ ]:
import pandas as pd
import os
import base64
from multiprocessing import Pool
from tqdm import tqdm

# Assuming you have a DataFrame named 'final_df' with columns 'FILE_NAME' and 'PATH'
def process_row(row):
    if row['FILE_NAME'] != 'NAO':
        path_final = os.path.join(row['PATH'], row['FILE_NAME'])
        with open(path_final, 'rb') as f:
            encoded_string = base64.b64encode(f.read()).decode()  # Decode to get string instead of bytes
            return encoded_string
    else:
        return None

def process_batch(batch):
    return [process_row(row) for _, row in batch.iterrows()]

# Number of workers
num_workers = 50

# Batch size for processing
batch_size = 1000

# Split DataFrame for parallel processing
chunks = [final_df.iloc[i:i+batch_size] for i in range(0, len(final_df), batch_size)]

# Function to process batches
def process_batches(chunks):
    results = []
    for chunk in tqdm(chunks, desc="Processing batches"):
        results.extend(process_batch(chunk))
    return results

# Initialize Pool of workers
pool = Pool(processes=num_workers)

# Process chunks in parallel with tqdm progress bar
results = pool.map(process_batches, chunks)

# Flatten the results list
results = [item for sublist in results for item in sublist]

# Close the pool to free resources
pool.close()
pool.join()

# Add the base64 data to the DataFrame
final_df['base64_file'] = results

In [ ]:
import pandas as pd
import json

# Assuming you have a DataFrame named 'df'

# Split the DataFrame into three parts
num_rows = len(final_df)
chunk_size = num_rows // 6

chunks = [final_df.iloc[i:i+chunk_size] for i in range(0, num_rows, chunk_size)]

# Save each chunk to a JSON file
for i, chunk in enumerate(chunks):
    chunk.to_json(f"chunk_{i+1}.json", orient="records")

In [ ]:
import pandas as pd
import os
import base64
from multiprocessing import Pool
from tqdm import tqdm

# Assuming you have a DataFrame named 'final_df' with columns 'FILE_NAME' and 'PATH'
def process_row(row):
    if row['FILE_NAME'] != 'NAO':
        path_final = os.path.join(row['PATH'], row['FILE_NAME'])
        print(path_final)
        caminhoArquivo = path_final
        with open(caminhoArquivo, 'rb') as f:
            encoded_string = base64.b64encode(f.read()).decode()  # Decode to get string instead of bytes
            return encoded_string
    else:
        return None
    
# Number of workers
num_workers = 20

# Split DataFrame for parallel processing
chunks = [final_df.iloc[i:i+len(final_df)//num_workers] for i in range(0, len(final_df), len(final_df)//num_workers)]

# Initialize Pool of workers
pool = Pool(processes=num_workers)

# Process chunks in parallel with tqdm progress bar
results = []
with tqdm(total=len(chunks)) as pbar:
    for chunk_result in pool.imap_unordered(process_row, chunks):
        results.extend(chunk_result)
        pbar.update(1)  # Update progress bar

# Close the pool to free resources
pool.close()
pool.join()

# Add the base64 data to the DataFrame
final_df['base64_file'] = results

In [ ]:
final_df.columns

In [ ]:
import pandas as pd
import os
import base64
from multiprocessing import Pool

# Assuming you have a DataFrame named 'final_df' with columns 'FILE_NAME' and 'PATH'
def process_row(row):
    if row['FILE_NAME'] != 'NAO':
        path_final = os.path.join(row['PATH'], row['FILE_NAME'])
        print(path_final)
        caminhoArquivo = path_final
        with open(caminhoArquivo, 'rb') as f:
            encoded_string = base64.b64encode(f.read())
            return encoded_string
    else:
        return None
    
# Number of workers
num_workers = 20

# Split DataFrame for parallel processing
chunks = [final_df.iloc[i:i+len(final_df)//num_workers] for i in range(0, len(final_df), len(final_df)//num_workers)]

# Initialize Pool of workers
pool = Pool(processes=num_workers)

# Process chunks in parallel
results = pool.map(process_row, [chunk for chunk in chunks])

# Close the pool to free resources
pool.close()
pool.join()

# Combine results into a single list
base64_data = []
for result in results:
    base64_data.extend(result)

# Add the base64 data to the DataFrame
final_df['base64_file'] = base64_data



In [ ]:
import os
import base64

final_df['base64_file'] = pd.series([])

for index, row in final_df.iterrows():
    if row['FILE_NAME'] != 'NAO':
        path_final = os.path.join(row['PATH'], row['FILE_NAME'])
        print(path_final)
        caminhoArquivo = path_final
        with open(caminhoArquivo, 'rb') as f:
            arquivo_bytes = f.read()
            arquivo_base64 = base64.b64encode(arquivo_bytes).decode()
        print(arquivo_base64)
        

In [ ]:
print(len(filtered_df))
print(len(df2))
print(len(final_df))

In [ ]:
final_df

In [ ]:
filtered_df.to_csv('dados.csv',index=None)

In [ ]:
final_df['FILE_NAME']

In [ ]:
import os
listDir = os.listdir("E:\RAIA_DROGASIL\IMAGENS\FINAL")

In [ ]:
dflist = pd.DataFrame({'col':listDir})

In [ ]:
dflist

In [ ]:
df_migrate = final_df[final_df['FILE_NAME'].isin(dflist['col'])]

In [ ]:
df_migrate